## Introduction:
Being anonymous over the internet can sometimes make people say nasty things that they normally would not in real life. Let's filter out the hate from our platforms one comment at a time.

## Objective:
To do exploratory data analysis for toxic comment classification and build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate better than Perspective’s current models. We will be using a dataset of comments from Wikipedia’s talk page edits. Improvements to the current model will hopefully help online discussion become more productive and respectful.

Disclaimer: the dataset for this competition contains text that may be considered profane, vulgar, or offensive.

## Data Overview:
The dataset here is from wiki corpus dataset which was rated by human raters for toxicity. The corpus contains 63M comments from discussions relating to user pages and articles dating from 2004-2015.

Different platforms/sites can have different standards for their toxic screening process. Hence the comments are tagged in the following five categories

toxic
severe_toxic
obscene
threat
insult
identity_hate

In [ ]:
#import required packages
import pandas as pd
import numpy as np

import re
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.utils import resample

In [ ]:
#importing the training dataset
train_data_or = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
train_data_or.head()

In [ ]:
#importing the testing dataset
test_data = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_data.head()

In [ ]:
#Dropping the unecessary features
train_data_or.drop(['id'],axis=1,inplace=True)
# test_data.drop(['id'],axis=1,inplace=True)

In [ ]:
#checking the shape of the dataset
train_data_or.shape

In [ ]:
x=train_data_or.iloc[:,2:].sum()

#marking comments without any tags as "clean"
rowsums=train_data_or.iloc[:,2:].sum(axis=1)
train_data_or['clean']=(rowsums==0)

#count number of clean entries
train_data_or['clean'].sum()
print("Total comments = ",len(train_data_or))
print("Total clean comments = ",train_data_or['clean'].sum())
print("Total tags =",x.sum())

*  We can see the the class is imbalance and the dataset is huge. So inorder to faster training and balance the dataset, we have downsampled the majority class.

In [ ]:
df_majority = train_data_or[train_data_or.clean==True]
df_minority = train_data_or[train_data_or.clean==False]

df_majority_downsampled = resample(df_majority,
                                  replace=False,
                                  n_samples=10000,
                                  random_state=123)

train_data = pd.concat([df_majority_downsampled,df_minority])

In [ ]:
train_data.shape

In [ ]:
#checking the distribution of tag over the dataset
train_data.iloc[:,1:].sum()

In [ ]:
#checking for the null or missing value in the dataset
train_data.isna().sum()

In [ ]:
def get_comment_type(row):
     for c in train_data.iloc[:,1:]:
        if row[c]==1:
            return c

comment_type = train_data.apply(get_comment_type, axis=1)
train_data['comment_type'] = comment_type

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='comment_type', data=train_data)
plt.xlabel("Type of comment")
plt.ylabel("Number of comment")
plt.show()

In [ ]:
train_data = train_data.fillna(value=np.nan)
train_data = train_data.fillna(value='safe')

In [ ]:
plt.figure(figsize=(10,5))

ax = (train_data.comment_type.value_counts()/len(train_data)*100).sort_index().plot(kind="bar", rot=0)
ax.set_yticks(np.arange(0, 110, 10))

for p in ax.patches:
    ax.annotate('{:.2f}%'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))
    
plt.xlabel("Type of comment")
plt.ylabel("Number of comment")
plt.show()

In [ ]:
#displaying the example from each type of toxic comment

print('\033[1m' + 'TOXIC' + '\033[0m')
print(train_data['comment_text'][train_data['toxic']==1].iloc[1])
print(' ')

print('******************************************************************')

print('\033[1m' + 'SEVERE TOXIC' + '\033[0m')
print(train_data['comment_text'][train_data['severe_toxic']==1].iloc[1])
print(' ')

print('******************************************************************')

print('\033[1m' + 'OBSCENE' + '\033[0m')
print(train_data['comment_text'][train_data['obscene']==1].iloc[1])
print(' ')

print('******************************************************************')

print('\033[1m' + 'THREAT' + '\033[0m')
print(train_data['comment_text'][train_data['threat']==1].iloc[0])
print(' ')

print('******************************************************************')

print('\033[1m' + 'INSULT' + '\033[0m')
print(train_data['comment_text'][train_data['insult']==1].iloc[1])
print(' ')

print('******************************************************************')

print('\033[1m' + 'IDENTITY HATE' + '\033[0m')
print(train_data['comment_text'][train_data['identity_hate']==1].iloc[1])

## Wordclouds - Frequent words:
Now, let's take a look at words that are associated with these classes.

Chart Desc: The visuals here are word clouds (ie) more frequent words appear bigger

In [ ]:
stopword=set(STOPWORDS)


#wordcloud for clean comments
subset=train_data[train_data.clean==True]
text=subset.comment_text.values
wc= WordCloud(background_color="black",max_words=2000,stopwords=stopword)
wc.generate(" ".join(text))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.title("Words frequented in Clean Comments", fontsize=20)
plt.imshow(wc.recolor(colormap= 'viridis' , random_state=17), alpha=0.98)
plt.show()

In [ ]:
#wordcloud for clean comments
subset=train_data[train_data.toxic==1]
text=subset.comment_text.values
wc= WordCloud(background_color="black",max_words=4000,stopwords=stopword)
wc.generate(" ".join(text))
plt.figure(figsize=(20,10))
plt.subplot(221)
plt.axis("off")
plt.title("Words frequented in Toxic Comments", fontsize=20)
plt.imshow(wc.recolor(colormap= 'gist_earth' , random_state=244), alpha=0.98)

#Severely toxic comments
plt.subplot(222)
subset=train_data[train_data.severe_toxic==1]
text=subset.comment_text.values
wc= WordCloud(background_color="black",max_words=2000,stopwords=stopword)
wc.generate(" ".join(text))
plt.axis("off")
plt.title("Words frequented in Severe Toxic Comments", fontsize=20)
plt.imshow(wc.recolor(colormap= 'Reds' , random_state=244), alpha=0.98)

#Threat comments
plt.subplot(223)
subset=train_data[train_data.threat==1]
text=subset.comment_text.values
wc= WordCloud(background_color="black",max_words=2000,stopwords=stopword)
wc.generate(" ".join(text))
plt.axis("off")
plt.title("Words frequented in Threatening Comments", fontsize=20)
plt.imshow(wc.recolor(colormap= 'summer' , random_state=2534), alpha=0.98)

#insult
plt.subplot(224)
subset=train_data[train_data.insult==1]
text=subset.comment_text.values
wc= WordCloud(background_color="black",max_words=2000,stopwords=stopword)
wc.generate(" ".join(text))
plt.axis("off")
plt.title("Words frequented in insult Comments", fontsize=20)
plt.imshow(wc.recolor(colormap= 'Paired_r' , random_state=244), alpha=0.98)

plt.suptitle("WordCloud of Toxic Words", fontsize=30)   
plt.tight_layout()
plt.show()

In [ ]:
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}

In [ ]:
#defining a function to clean the data
def clean_text(text):
    
    text = text.lower()
    text = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', '', text) # clean url
    text = re.sub(r'#(\w+)', '', text)   # clean hashes
    text = re.sub(r'@(\w+)', '', text)   # clean @
    text = re.sub(r'<[^>]+>', '', text)  # clean tags
    text = re.sub(r'\d+', '', text)      # clean digits
    text = re.sub(r'[,!@\'\"?\.$%_&#*+-:;]', '', text)   # clean punctuation
    text = [APPO[word] if word in APPO else word for word in text.split()]  #
    
    return text

In [ ]:
#applying the cleantext function to bothe train and test data
train_data['comment_text'] = train_data['comment_text'].apply(clean_text)
test_data['comment_text'] = test_data['comment_text'].apply(clean_text)

In [ ]:
train_x = train_data.iloc[:,0]
train_y = train_data.iloc[:,1:7]

train_y = np.array(train_y)

In [ ]:
#Splitting the data into train and validation
train_x, val_x, train_y, val_y = train_test_split(train_x,train_y, test_size=0.2, random_state=1)

In [ ]:
tokenizer = Tokenizer(num_words = 100000, oov_token='<oov>')
tokenizer.fit_on_texts(train_data.comment_text)

In [ ]:
traning_sequences = tokenizer.texts_to_sequences(train_x)
maxlen = max([len(x) for x in np.array(traning_sequences)])
training_padded = pad_sequences(traning_sequences, maxlen = maxlen,
                                padding = 'pre',
                                truncating='pre')

In [ ]:
validation_sequences = tokenizer.texts_to_sequences(val_x)
validation_padded = pad_sequences(validation_sequences, maxlen = maxlen,
                                padding = 'pre',
                                truncating='pre')

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [ ]:
embeddings_index = {}
glovefile = open('../input/glove6b200d/glove.6B.200d.txt','r',encoding='utf-8')
for line in tqdm(glovefile):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
#     coefs.shape
    embeddings_index[word] = coefs
glovefile.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
#creating embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(tokenizer.word_index)+1, 200))
for words, index in tqdm(tokenizer.word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
#Building a bidirectional LSTM model
model = tf.keras.Sequential([tf.keras.layers.Embedding(input_dim = vocab_size,output_dim = 200,weights = [embedding_matrix],input_length = maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True)),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation='sigmoid')])

In [ ]:
#Summary of the model
model.summary()

In [ ]:
#Compiling and running the model
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(training_padded,train_y, epochs = 2, validation_data=(validation_padded, val_y), batch_size = 32)

In [ ]:
#Predicting on the validation data
predicted = model.predict(validation_padded)
labels = (predicted > 0.5).astype(np.int)

In [ ]:
def get_toxictype(i,act_or_pred):
    
    l=[]
    count=0
    
    if sum(act_or_pred[:10][i])==0:
        l.append('safe comment')
        
    else:
        for j in range(len(act_or_pred[:10][i])):
            if act_or_pred[:10][i][j]==1 and count == 0:
                l.append('toxic')
            elif act_or_pred[:10][i][j]==1 and count == 1:
                l.append('severe_toxic')
            elif act_or_pred[:10][i][j]==1 and count == 2:
                l.append('obscene')
            elif act_or_pred[:10][i][j]==1 and count == 3:
                l.append('threat')
            elif act_or_pred[:10][i][j]==1 and count == 4:
                l.append('insult')
            elif act_or_pred[:10][i][j]==1 and count == 5:
                l.append('identity_hate')
            
            count=count+1
            
    return l

### Displaying the actual  predicted type of comment

In [ ]:
for i,j in enumerate(val_x.index[:10]):
    print('')
    print('\033[1m' + 'Predicted type of comment:' + '\033[0m', get_toxictype(i,labels))
    print('\033[1m' + 'Actual type of comment:' + '\033[0m', get_toxictype(i,val_y))
    print('\033[1m' + 'Comment: ' + '\033[0m',train_data_or.iloc[j,0])
    print('')
    print('****************************************************************************')

### Predicting the result on the Test dataset

In [ ]:
testing_sequences = tokenizer.texts_to_sequences(test_data.comment_text)
test_padded = pad_sequences(testing_sequences, maxlen = maxlen,
                                padding = 'pre',
                                truncating='pre')

In [ ]:
predicted = model.predict(test_padded, batch_size = 200)
predict = np.hstack((test_data.id[:, np.newaxis], predicted))

In [ ]:
subm = pd.DataFrame(predict, columns = ['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
subm.to_csv('subm.csv', index = False)